This colab contains code for creating a correlation clustering problem defined by two weight matrices, W_plus and W_minus. You should add code to implement an approximation algorithm using semidefinite programming and randomized rounding, as described in [Williamson and Shmoys](https://www.designofapproxalgs.com/book.pdf) section 6.4.

# Construct weight matrices

In [1]:
# Fetch and import libraries
import picos as pc
import cvxopt as cvx
import cvxopt.lapack
from scipy.linalg import cholesky
import numpy as np
import itertools as it

In [5]:
# Fetch data
!mkdir data
!curl https://raw.githubusercontent.com/rasmus-pagh/apx/main/data/denmark-0.6.txt -o ./data/denmark-0.6.txt
!curl https://raw.githubusercontent.com/rasmus-pagh/apx/main/data/learning-0.6.txt -o ./data/learning-0.6.txt
!curl https://raw.githubusercontent.com/rasmus-pagh/apx/main/data/copenhagen-0.5.txt -o ./data/copenhagen-0.5.txt


mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  103k  100  103k    0     0   368k      0 --:--:-- --:--:-- --:--:--  369k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  162k  100  162k    0     0  1317k      0 --:--:-- --:--:-- --:--:-- 1321k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  323k  100  323k    0     0  1220k      0 --:--:-- --:--:-- --:--:-- 1217k


There are three data files containing [GloVe](https://nlp.stanford.edu/projects/glove/) vectors, whose dot products measure the similarity between words. The matrix W_plus is defined as dot products of such vectors, while W_minus is constant, equaling the average in W_plus.

In [124]:
# Choose one of the three data files
filename = "./data/" + ['learning-0.6.txt', 'copenhagen-0.5.txt', 'denmark-0.6.txt'][2]

# Read vectors and construct matrices
with open(filename, 'r') as f:
  feature_vectors = []
  words = []
  for line in f:
    word, vector = line.split(';')
    words.append(word)
    vector = [ float(x) for x in vector.split(',') ]
    feature_vectors.append(vector)
  n = len(words)
  feature_vectors = np.array(feature_vectors)
  W_plus = np.dot(feature_vectors, np.transpose(feature_vectors))
  W_minus = np.ones(shape=(n,n)) * np.average(W_plus)

51


# Correlation clustering

Here you can implement your approximation algorithm. It may be helpful to consult the [implementation](https://colab.research.google.com/drive/1Rhe0kra6mqt5VHc2uTlNzJ_JC6kpG8nA?usp=sharing) of an approximation algorithm for Max Cut. Your implementation must:
- Define and solve the semidefinite programming relaxation
- Output the upper bound on OPT given by the relaxation
- Output the expected value of the objective function with a random 4-clustering
- Output the value of the best 4-clustering found using randomized rounding (say, in 100 trials), and the words placed in each cluster.

If you experience problems with convergence (optimizer not terminating) don't worry about it.


In [144]:
correlation_clustering = pc.Problem()

# Define variables and constants
X = pc.SymmetricVariable('X',(n,n))
W_p = pc.Constant('W_p', W_plus)
W_m = pc.Constant('W_m', W_minus)
ones = pc.Constant('ones', np.ones((n,n)))

# Define objective and constraints
correlation_clustering.set_objective('max', (W_p | X) + (W_m | (ones - X)))
correlation_clustering.add_constraint(pc.maindiag(X) == 1)
correlation_clustering.add_constraint(X >= 0)
correlation_clustering.add_constraint(X >> 0)

# Solve the problem
correlation_clustering.solve(solver='cvxopt', verbosity=1)
V = cholesky(X.value + 1e-3 * np.identity(n))

           PICOS 2.4.17            
Problem type: Semidefinite Program.
Searching a solution strategy for CVXOPT.
Solution strategy:
  1. ExtraOptions
  2. CVXOPTSolver
Applying ExtraOptions.
Building a CVXOPT problem instance.
Starting solution search.
-----------------------------------
 Python Convex Optimization Solver 
    via internal CONELP solver     
-----------------------------------
     pcost       dcost       gap    pres   dres   k/t
 0: -2.1196e+01 -2.1196e+01  8e+03  5e+01  2e+01  1e+00
 1: -2.8263e+01 -2.8344e+01  2e+03  3e+01  1e+01  5e-01
 2: -5.5176e+01 -5.4808e+01  8e+02  8e+00  3e+00  5e-01
 3: -7.2168e+01 -7.1971e+01  4e+02  4e+00  2e+00  3e-01
 4: -9.1143e+01 -9.1095e+01  2e+02  2e+00  7e-01  8e-02
 5: -9.8230e+01 -9.8206e+01  8e+01  1e+00  4e-01  4e-02
 6: -1.0469e+02 -1.0468e+02  3e+01  4e-01  2e-01  1e-02
 7: -1.0637e+02 -1.0637e+02  2e+01  2e-01  9e-02  9e-03
 8: -1.0780e+02 -1.0780e+02  1e+01  1e-01  5e-02  5e-03
 9: -1.0862e+02 -1.0862e+02  6e+00  7e-02  3

In [137]:
print(f"Upper bound of OPT given by the relaxation: {correlation_clustering.value}")
print(f"Expected value of random 4-clustering: {1/4 * np.sum(W_plus) + 3/4 * np.sum(W_minus)}")

Upper bound of OPT given by the relaxation: 1629.8608433121885
Expected value of random 4-clustering: 1520.0115731912529


In [253]:
partition_weights = []
partitions = []
repetitions = 10**5

# Perform randomized rounding many times, compute the cluster weight each time
for _ in range(repetitions):
  # Generate two random hyperplanes and compute dot product with V
  R = np.random.normal(size = (2, n))
  R_dot_V = np.dot(R, V)
  
  # Compute the vertices of each cluster
  P = [(R_dot_V[0] >= 0) * (R_dot_V[1] >= 0), 
       (R_dot_V[0] >= 0) * (R_dot_V[1] < 0), 
       (R_dot_V[0] < 0) * (R_dot_V[1] >= 0),
       (R_dot_V[0] < 0) * (R_dot_V[1] < 0)]
  
  # Compute the words for each cluster
  partitions.append([np.array(words)[c] for c in P])
  
  # Compute the edges within clusters (plus_edges) and between clusters (minus_edges)
  plus_edges = np.sum([np.outer(c, c) for c in P], axis=0)
  minus_edges = 1 - plus_edges  
  
  # Compute the cluster weight
  partition_weights.append(np.sum(W_plus * plus_edges + W_minus * minus_edges))
  

print(f"Cluster weight found by randomized rounding: {max(partition_weights)} (best) and {min(partition_weights)} (worst)")
partition = partitions[np.argmax(partition_weights)]
print(f"Clusters of best 4-clustering found by randomized rounding:")
for i in range(4):
  print(f"\tCluster {i+1}: {partition[i]}")
  



Cluster weight found by randomized rounding: 1623.067219764774 (best) and 1515.6005888133845 (worst)
Clusters of best 4-clustering found by randomized rounding:
	Cluster 1: ['brisbane' 'newcastle' 'melbourne' 'nz' 'australia' 'wales' 'london'
 'sydney' 'cardiff' 'england']
	Cluster 2: ['switzerland' 'belgium' 'finland' 'slovakia' 'holland' 'zealand' 'russia'
 'bulgaria' 'czech' 'serbia' 'denmark' 'austria' 'lithuania' 'germany'
 'slovenia' 'italy' 'norway' 'ireland' 'croatia' 'brazil' 'spain' 'greece'
 'sweden' 'netherlands' 'latvia' 'romania' 'poland' 'europe' 'hungary'
 'iceland' 'belarus' 'azerbaijan' 'kazakhstan' 'estonia' 'morocco']
	Cluster 3: ['netherland' 'englan']
	Cluster 4: ['belanda' 'jerman' 'thailand' 'portugal']
